In [1]:
using StatisticalRethinking, CmdStan
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
first(df2, 5)

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real<lower=0> h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=0,upper=250> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchains);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.1s.jls", chn)
chn2 = deserialize("m4.1s.jls")

describe(chn2)


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/heights.stan will be updated.


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.038, 0.033, 0.038, 0.033) seconds, 0.14 seconds total
Sampling took (0.047, 0.044, 0.052, 0.056) seconds, 0.20 seconds total

                Mean     MCSE  StdDev    5%   50%   95%  N_Eff  N_Eff/s    R_hat
lp__            -892  2.5e-02     1.0  -894  -891  -891   1626     8195  1.0e+00
accept_stat__   0.91  1.7e-03    0.11  0.68  0.96   1.0   4371    22032  1.0e+00
stepsize__      0.85  5.1e-02   0.072  0.73  0.89  0.92    2.0       10  9.2e+13
treedepth__      1.9  1.1e-01    0.54   1.0   2.0   3.0     26      132  1.1e+00
n_leapfrog__     3.8  4.0e-01     2.5   1.0   3.0   7.0     38      193  1.0e+00
divergent__     0.00      nan    0.00  0.00  0.00  0.00    nan      nan      nan
energy__         892  3.8e-02     1.4   891   892

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 1 columns
│ Row │ parameters │ mean    │ std      │ naive_se   │ mcse       │ ess     │
│     │ Symbol     │ Float64 │ Float64  │ Float64    │ Float64    │ Any     │
├─────┼────────────┼─────────┼──────────┼────────────┼────────────┼─────────┤
│ 1   │ mu         │ 154.611 │ 0.413698 │ 0.00654113 │ 0.00638993 │ 3770.63 │
│ 2   │ sigma      │ 7.76567 │ 0.29539  │ 0.00467053 │ 0.00518002 │ 3334.06 │

Quantiles

│ Row │ parameters │ 2.5%    │ 25.0%   │ 50.0%   │ 75.0%   │ 97.5%   │
│     │ Symbol     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ mu         │ 153.825 │ 154.338 │ 154.603 │ 154.887 │ 155.428 │
│ 2   │ sigma      │ 7.21063 │ 7.55912 │ 7.75863 │ 7.9609  │ 8.35406 │


end of m4.1s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*